# Deploy HLS Foundation Model into Vertex AI 

In [ ]:
%%writefile requirements.txt
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
google-cloud-storage>=1.26.0,<2.0.0dev
     

In [ ]:
# Clone github with inference code for HLS foundation model
! git clone https://github.com/nasa-impact/hls-foundation-vertexai.git

In [ ]:
# Navigate inside hls-foundation-vertexai folder
import os
os.chdir('hls-foundation-vertexai')

In [ ]:
# Required in Docker serving container
! pip3 install -U  -r requirements.txt -q --user

# For local FastAPI development and running
! pip3 install -U  "uvicorn[standard]>=0.12.0,<0.14.0" fastapi~=0.63 -q --user

# Vertex SDK for Python
! pip3 install --upgrade --quiet  google-cloud-aiplatform --user
     

In [ ]:
# List configuration
! gcloud config list

In [ ]:
# List projects
! gcloud projects list

In [ ]:
# Replace <project-id> with one of the project ids from above.
! gcloud builds submit --region=us-central1 --tag=us-central1-docker.pkg.dev/<project-id>/hls-foundation-vertexai/inference

## Use python to deploy HLS foundation model to vertexai endpoint 

In [ ]:
from google.cloud import aiplatform

In [ ]:
model = aiplatform.Model.upload(
    display_name='hls-inference',
    serving_container_image_uri="us-central1-docker.pkg.dev/msfc-st11-impact-3a77f55d/hls-foundation-vertexai/inference",
)

In [ ]:
model


In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4", accelerator_type='NVIDIA_TESLA_V100', accelerator_count=1)

In [ ]:
%%writefile test.json
{
  "instances":{"date":"2023-08-13","bounding_box":[-156.81605703476012,20.69675592885614,-156.41605703476014,21.096755928856137], "model_id": "burn_scars"},
}


In [ ]:
# Get inference from the deployed endpoint. Copy over the endpoint id from above and replace <ENDPOINT_ID>, and project id from about to replace <PROJECT_ID>
! export ENDPOINT_ID=<ENDPOINT_ID>; export PROJECT_ID=<PROJECT_ID>; export INPUT_DATA_FILE="test.json"; curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-central1/endpoints/${ENDPOINT_ID}:predict \
-d "@${INPUT_DATA_FILE}"
